<a href="https://colab.research.google.com/github/sunnygong369/Covisitation-Matrix/blob/main/MOVIE_MATRIX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Settings**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Data source:** https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?select=ratings.csv

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_ratings.csv') #load the csv file --> the "history table"
df = df[["userId", "movieId", "timestamp"]]
df = df.reset_index(drop = True)
df.head(10)

,userId,movieId,timestamp
0,1,110,1425941529
1,1,147,1425942435
2,1,858,1425941523
3,1,1221,1425941546
4,1,1246,1425941556
5,1,1968,1425942148
6,1,2762,1425941300
7,1,2918,1425941593
8,1,2959,1425941601
9,1,4226,1425942228


### **Rearranging the dataset**

*   **Sorting by timestamp [to make order matter]**

In [ ]:
df_grp = df.groupby('userId') #group by "userId"

df_grp_srt = df_grp.apply(lambda x: x.sort_values('timestamp')) #sorting each group by timestamp (ordered)

df_grp_srt

userId  movieId   timestamp
userId                                    
1      6            1     2762  1425941300
       13           1    54503  1425941313
       26           1   112552  1425941336
       23           1    96821  1425941382
       11           1     5577  1425941397
...               ...      ...         ...
5108   493918    5108     2648  1447443778
       493912    5108      318  1447443880
       493914    5108     1219  1447443955
       493911    5108      296  1447443957
5109   493924    5109     1345   950884803

[493925 rows x 3 columns]

In [ ]:
#quick check on timestamp sorting:
df_grp_srt[df_grp_srt['userId'] == 99] #replace 99 with any number

*   **Filtering out the timestamp column**

In [ ]:
df_new = zip(df_grp_srt['userId'].tolist(),df_grp_srt['movieId'].tolist()) #put 'userId' and 'movieId' to list
df_new = pd.DataFrame(df_new)
df_new.columns = ['User ID', 'Movie ID'] #rename the columns
df_new.index.name = 'Index'

df_new

,User ID,Movie ID
Index,,
0,1,2762
1,1,54503
2,1,112552
3,1,96821
4,1,5577
...,...,...
493920,5108,2648
493921,5108,318
493922,5108,1219


------------------------------------------------------------------------------
*   **Step abandoned [crashed for trying to display entries in "user1 item1 item2 item3 ... itemN" format]**
------------------------------------------------------------------------------


In [ ]:
# aimed to arrange data in the format as such: "user1 item1 item2 item3 ... itemN" for all the rows
#it will crash --> don't run
#it will crash --> don't run
#it will crash --> don't run

df_new_wide = df_new.pivot_table(index='User ID', columns='Index', values='Movie ID', aggfunc='first') 

df_new_wide

*   **Rearranging the entries**

In [ ]:
df_new1 = df_new.groupby('User ID')['Movie ID'].agg(list).reset_index() #take the "User ID" and "Movie ID" columns to create the users' history table
df_new1

,User ID,Movie ID
0,1,"[2762, 54503, 112552, 96821, 5577, 92439, 4878..."
1,2,"[780, 32, 648, 25, 141, 5, 786, 260, 1356, 788..."
2,3,"[3101, 527, 480, 500, 858, 1097, 1270, 1092, 4..."
3,4,"[4014, 4896, 1210, 1221, 415, 4084, 1097, 1431..."
4,5,"[1230, 157, 1302, 1270, 7, 3255, 3114, 3079, 3..."
...,...,...
5104,5105,"[1285, 1694, 1393, 1704, 1242, 36, 778, 866, 3..."
5105,5106,"[2174, 2291, 236, 1080, 1258, 315, 2011, 784, ..."
5106,5107,"[797, 3321, 4178, 4685, 4391, 4640, 4341, 4848..."
5107,5108,"[2858, 1222, 1258, 356, 293, 1246, 26528, 2692..."


In [ ]:
df_new2 = df_new1["Movie ID"] #select "Movie ID" --> to creat pairs at the next stage
df_new2

0       [2762, 54503, 112552, 96821, 5577, 92439, 4878...
1       [780, 32, 648, 25, 141, 5, 786, 260, 1356, 788...
2       [3101, 527, 480, 500, 858, 1097, 1270, 1092, 4...
3       [4014, 4896, 1210, 1221, 415, 4084, 1097, 1431...
4       [1230, 157, 1302, 1270, 7, 3255, 3114, 3079, 3...
                              ...                        
5104    [1285, 1694, 1393, 1704, 1242, 36, 778, 866, 3...
5105    [2174, 2291, 236, 1080, 1258, 315, 2011, 784, ...
5106    [797, 3321, 4178, 4685, 4391, 4640, 4341, 4848...
5107    [2858, 1222, 1258, 356, 293, 1246, 26528, 2692...
5108                                               [1345]
Name: Movie ID, Length: 5109, dtype: object

*   **Finding all the pairs from the entries** (time: 3s + 2s from the last run)

In [ ]:
from nltk import bigrams
bigrams_list = [list(bigrams(i)) for i in df_new2] #finding all the pairs
bigrams_list

bigrams_list = [[str(i) for i in sublist] for sublist in bigrams_list] #making all the pairs to string (for counting purposes)
bigrams_list

In [ ]:
bigrams_list = [[i.replace('(', '').replace(')', '') for i in sublist] for sublist in bigrams_list] #remove all brackets
bigrams_list

*   **Counting all the pairs**

In [ ]:
from collections import Counter

all_pairs = [i for sublist in bigrams_list for i in sublist] #list of lists unpacked

pairs_counter = Counter(all_pairs) #counting unique pairs from "history" (i.e. ordered entries)
pairs_counter

### **Constructing a [blank] Covisitation Matrix**

*   **Unique IDs for labels**

In [ ]:
unique_val = pd.DataFrame(pd.unique(df_new2.explode())) #unpack and collect unique values -> then make it a df
unique_val_srt = unique_val[0].sort_values().astype(str).tolist() #sorting df by unique values -> making them strings (for labeling purposes) in list
unique_val_srt

*   **Blank matrix filled with zeros**

In [ ]:
co_matrix = np.zeros((len(unique_val_srt), len(unique_val_srt)), dtype=int) #a blank covisitation matrix created
co_matrix = pd.DataFrame(co_matrix) #converting matrix to df
co_matrix.index = unique_val_srt #labeling the rows with unique IDs
co_matrix.columns = unique_val_srt #labeling the columns ...
co_matrix

------------------------------------------------------------------------------
*   **Step abandoned [crashed for trying to display all the pairs in matrix]**
------------------------------------------------------------------------------


In [ ]:
#display all the pairs in the matrix
#don't run --> it will crash
#don't run --> it will crash
#don't run --> it will crash

for i in co_matrix.index: 
  for j in co_matrix.columns:
    co_matrix[i][j] = (str(i)+str(j))

co_matrix

### **Constructing the Covisitation Matrix** 
**Filling the Covisitation Matrix** (time: 30s from the last run)

In [ ]:
for key, value in dict(pairs_counter).items():
   a = key.split(",")[0].strip()
   b = key.split(",")[1].split(":")[0].strip()
   c = value

   co_matrix.loc[a,b] = c ###### it may have writen c to cell [a, b], but it is not showing in the table below

co_matrix


,1,2,3,4,5,6,7,8,9,10,...,175475,175579,175589,175773,175775,175777,175779,175945,176165,176271
1,0,2,0,0,2,1,0,0,0,6,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,1,1,3,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,28,11,14,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,19,1,0,16,14,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175777,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
175779,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
175945,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
176165,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#for checking only (i.e. the top 50x50 matrix)
co_matrix[:50][:50]

In [ ]:
#for checking only (i.e. checking the values of a, b, c) 

for key, value in dict(pairs_counter).items():
   a = key.split(",")[0].strip()
   b = key.split(",")[1].split(":")[0].strip()
   c = value

   print (a, b, c)

### **Predicting**

*   **Normalizing the matrix (could be skipped)** (time: 2s from the last run)




In [ ]:
from sklearn.preprocessing import normalize #TO CONFIRM (How it should be normalized?) ALSO, cosine_similarity with or without the normalizing the matrix seem to give the same result

co_matrix_norm = pd.DataFrame(normalize(co_matrix))
co_matrix_norm.index = unique_val_srt #label the rows with the list of unique item ID
co_matrix_norm.columns = unique_val_srt #label the columns ...

co_matrix_norm

,1,2,3,4,5,6,7,8,9,10,...,175475,175579,175589,175773,175775,175777,175779,175945,176165,176271
1,0.000000,0.012077,0.000000,0.000000,0.012077,0.006038,0.000000,0.0,0.000000,0.036231,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.069007,0.000000,0.000000,0.023002,0.023002,0.069007,0.023002,0.0,0.000000,0.023002,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.043602,0.000000,0.000000,0.000000,0.610429,0.239811,0.305215,0.0,0.021801,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.129099,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.463277,0.024383,0.000000,0.390128,0.341362,0.0,0.000000,0.024383,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175777,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
175779,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
175945,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
176165,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


*   **Cosine_similarity_scores (with normalized matrix)** (time: 2m from the last run)







In [ ]:
from sklearn.metrics.pairwise import cosine_similarity #[cosine_similarity (with normalized matrix)]

cos_score_df = pd.DataFrame(cosine_similarity(co_matrix_norm)) #compute similarity score with cosine_similarity (without normalized the matrix)
cos_score_df.index = unique_val_srt #label the rows with the list of unique item ID
cos_score_df.columns = unique_val_srt #label the columns ...

cos_score_df

,1,2,3,4,5,6,7,8,9,10,...,175475,175579,175589,175773,175775,175777,175779,175945,176165,176271
1,1.000000,0.199041,0.069245,0.033521,0.077152,0.180646,0.080731,0.014290,0.036629,0.175743,...,0.0,0.0,0.006038,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.199041,1.000000,0.134896,0.044544,0.125633,0.164764,0.184516,0.058321,0.089548,0.146228,...,0.0,0.0,0.023002,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.069245,0.134896,1.000000,0.050661,0.528917,0.590083,0.567912,0.029480,0.122374,0.055332,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.033521,0.044544,0.050661,1.000000,0.116470,0.062167,0.078292,0.043644,0.058441,0.077279,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.077152,0.125633,0.528917,0.116470,1.000000,0.522006,0.625761,0.020607,0.055188,0.041743,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175777,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0
175779,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,1.0,0.0
175945,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0
176165,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [ ]:
new_user = np.zeros(len(co_matrix), dtype=int) #create a vector as a new user entry
random_index = np.random.randint(0, len(co_matrix)) #make a random index

new_user[random_index]=1 #generate 1 for a random-th row
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1
new_user[random_index]=1

len(new_user) == len(co_matrix)
new_user

array([0, 0, 0, ..., 0, 0, 0])

*   **Predicting (making recommendations) for a new user**

In [ ]:
A = new_user
B = cos_score_df.to_numpy()

out_asc = np.sort(np.matmul(A, B))
out_desc = out_asc[::-1] #sort the score in desc for top similarty scores
recommendation_all = [] 
candidates_only = []

for i in range(len(new_user)):
  if new_user[i] == 0:
    recommendation_all.append(out_desc[i])
    candidates_only.append(out_desc[i])
  if new_user[i] > 0: 
    recommendation_all.append(str(out_desc[i]) + ' implied preference') #mark those that are already preferred by the user

recommendation_all[:100] #view the top 100 candidates (including implied preference)
candidates_only[:100] #view the top 100 candidates

[1.0000000000000002,
 0.7071067811865476,
 0.7071067811865476,
 0.7071067811865476,
 0.5000000000000001,
 0.4082482904638631,
 0.316227766016838,
 0.31622776601683794,
 0.29095718698132306,
 0.2886751345948129,
 0.27735009811261463,
 0.2672612419124244,
 0.2672612419124244,
 0.23570226039551584,
 0.23570226039551584,
 0.223606797749979,
 0.21320071635561044,
 0.18898223650461363,
 0.18898223650461363,
 0.1825741858350554,
 0.1767766952966369,
 0.17334959899709632,
 0.17149858514250885,
 0.16770509831248429,
 0.16722501552266283,
 0.16222142113076254,
 0.158113883008419,
 0.15811388300841897,
 0.15811388300841897,
 0.15075567228888181,
 0.14433756729740646,
 0.14433756729740646,
 0.14359163172354766,
 0.14142135623730948,
 0.14142135623730948,
 0.14142135623730948,
 0.13867504905630731,
 0.13867504905630731,
 0.13867504905630731,
 0.13545709229571928,
 0.13470397652008123,
 0.1338965528032356,
 0.1336306209562122,
 0.13130643285972257,
 0.12909944487358058,
 0.12909944487358058,
 0.1270